In [231]:
import pandas as pd
import numpy as np


In [232]:
wsp_contract = pd.read_excel("../../Datasets/study-1-data.xlsx")



In [233]:
wsp_contract['Sales'] = wsp_contract[['Demand', 'Stock']].min(axis=1)

In [234]:
wsp_contract['Leftovers'] = wsp_contract['Stock'] - wsp_contract['Sales']

In [235]:
wsp_contract['Unmet_Demand'] = wsp_contract['Demand'] - wsp_contract['Sales']

In [236]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Unnamed: 5,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand
0,1,1,Manufacturer 1,122,8,NaN,Retailer 1,60,60,0,62
1,1,2,Manufacturer 1,144,7,NaN,Retailer 1,80,80,0,64
2,1,3,Manufacturer 1,57,7,NaN,Retailer 1,80,57,23,0
3,1,4,Manufacturer 1,149,9,NaN,Retailer 1,51,51,0,98
4,1,5,Manufacturer 1,135,8,NaN,Retailer 1,60,60,0,75


In [237]:
wsp_contract.drop(columns=["Unnamed: 5"], inplace=True)

In [238]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75


In [239]:
wsp_contract['Realized_Mfg_Profit'] = (wsp_contract['Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Stock']) 

In [240]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300


In [241]:
wsp_contract['Total_Revenue'] = wsp_contract['Stock'] * wsp_contract['Wholesale p.']

In [242]:
wsp_contract['Realized_Mfg_Profit_Share'] = wsp_contract.apply(lambda row: row['Realized_Mfg_Profit'] / row['Total_Revenue'] if row['Total_Revenue'] != 0 else 0, axis=1)

In [243]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000


In [244]:

wsp_contract['Optimal_Stock'] = wsp_contract['Demand'].mean()

In [245]:
wsp_contract['Under/Overstock_wrt_Optimal'] = wsp_contract['Stock'] - wsp_contract['Optimal_Stock']

In [246]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000,106.125,-46.125
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429,106.125,-26.125
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429,106.125,-26.125
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667,106.125,-55.125
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000,106.125,-46.125


In [247]:
potential_demand = np.arange(50, 151)
expected_sales = []

for index, row in wsp_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Stock'])

        cumulative_expected_sales += sales
        
        if sales == row['Stock']:
            cumulative_expected_sales += (150 - demand_realization) * row['Stock']
            break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

wsp_contract['Expected Sales'] = np.array(expected_sales)

In [248]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000,106.125,-46.125,59.455446
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429,106.125,-26.125,75.396040
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429,106.125,-26.125,75.396040
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667,106.125,-55.125,50.990099
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000,106.125,-46.125,59.455446
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,300,480,0.625000,106.125,-46.125,59.455446
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,0,0,0.000000,106.125,-106.125,0.000000
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,306,459,0.666667,106.125,-55.125,50.990099
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,0,0,0.000000,106.125,-106.125,0.000000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,300,480,0.625000,106.125,-46.125,59.455446


In [249]:
wsp_contract["Expected_Leftovers"] = wsp_contract["Optimal_Stock"] - wsp_contract["Expected Sales"]

In [250]:
wsp_contract['Expected Retailer Profit'] = wsp_contract['Expected Sales'] * wsp_contract['Wholesale p.'] - (3 * wsp_contract['Expected Sales']) #nasıl yapacaz kaça sattığını bilmiyoz

In [251]:
wsp_contract['Expected Mfg Profit'] = (wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Optimal_Stock'])

In [252]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000,106.125,-46.125,59.455446,46.669554,297.277228,530.625
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429,106.125,-26.125,75.396040,30.728960,301.584158,424.500
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429,106.125,-26.125,75.396040,30.728960,301.584158,424.500
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667,106.125,-55.125,50.990099,55.134901,305.940594,636.750
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000,106.125,-46.125,59.455446,46.669554,297.277228,530.625


In [253]:
wsp_contract['Total Expected Revenue'] = wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']

wsp_contract['Expected Mfg. Profit Share'] = wsp_contract.apply(lambda row: row['Expected Mfg Profit'] / row['Total Expected Revenue'] if row['Total Expected Revenue'] != 0 else 0, axis=1)

In [254]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Total Expected Revenue,Expected Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,480,0.625000,106.125,-46.125,59.455446,46.669554,297.277228,530.625,849.000,0.625000
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,560,0.571429,106.125,-26.125,75.396040,30.728960,301.584158,424.500,742.875,0.571429
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,560,0.571429,106.125,-26.125,75.396040,30.728960,301.584158,424.500,742.875,0.571429
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,459,0.666667,106.125,-55.125,50.990099,55.134901,305.940594,636.750,955.125,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,480,0.625000,106.125,-46.125,59.455446,46.669554,297.277228,530.625,849.000,0.625000
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,...,480,0.625000,106.125,-46.125,59.455446,46.669554,297.277228,530.625,849.000,0.625000
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,...,0,0.000000,106.125,-106.125,0.000000,106.125000,0.000000,849.000,1167.375,0.727273
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,...,459,0.666667,106.125,-55.125,50.990099,55.134901,305.940594,636.750,955.125,0.666667
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,...,0,0.000000,106.125,-106.125,0.000000,106.125000,0.000000,955.125,1273.500,0.750000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,...,480,0.625000,106.125,-46.125,59.455446,46.669554,297.277228,530.625,849.000,0.625000
